### SQuAD 데이터 입출력

SQuAD 데이터셋에 대한 모델 입출력은 아래와 같다.

`question`과 `text`를 구분자(`[CLS]`, `[SEP]`)로 구분하여 아래와 같이 `inputs`을 만든다

위 입력에 대한 정답인 `start_position`과 `end_position`을 인덱스로 나타낸다.

`inputs`의 10번째 토큰부터 12번째 토큰은 `['a', 'nice', 'puppet']`이며 `"Who was Jim Hensom"`에 대한 정답이다.

### Dataset 및 DataLoader 생성

In [1]:
import torch
from tqdm import tqdm, trange
from transformers import BertTokenizer, BertForQuestionAnswering, AdamW
from torch import nn
from torch.utils.data import DataLoader
from dataloader import SquadDataLoader
from dataset import SquadDataset

In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [3]:
train_dataset = SquadDataset('data', tokenizer, is_train=True)

cache file exists


In [9]:
train_dataloader = SquadDataLoader(train_dataset, batch_size=16, is_inference=False, shuffle=True)

In [10]:
for i, batch in enumerate(train_dataloader):
    if i > 10:
        break
    input_ids, input_mask, segment_ids, cls_index, p_mask, start_positions, end_positions = batch
    print(input_ids.shape, input_mask.shape, segment_ids.shape, cls_index.shape, p_mask.shape, start_positions.shape, end_positions.shape)
#     inputs = {
#         'input_ids': input_ids,
#         'token_type_ids': segment_ids,
#         'attention_mask': p_mask,
#     }
#     outputs = model(**inputs, start_positions=start_positions, end_positions=end_positions)
#     break

torch.Size([16, 384]) torch.Size([16, 384]) torch.Size([16, 384]) torch.Size([16]) torch.Size([16, 384]) torch.Size([16]) torch.Size([16])
torch.Size([16, 384]) torch.Size([16, 384]) torch.Size([16, 384]) torch.Size([16]) torch.Size([16, 384]) torch.Size([16]) torch.Size([16])
torch.Size([16, 384]) torch.Size([16, 384]) torch.Size([16, 384]) torch.Size([16]) torch.Size([16, 384]) torch.Size([16]) torch.Size([16])
torch.Size([16, 384]) torch.Size([16, 384]) torch.Size([16, 384]) torch.Size([16]) torch.Size([16, 384]) torch.Size([16]) torch.Size([16])
torch.Size([16, 384]) torch.Size([16, 384]) torch.Size([16, 384]) torch.Size([16]) torch.Size([16, 384]) torch.Size([16]) torch.Size([16])
torch.Size([16, 384]) torch.Size([16, 384]) torch.Size([16, 384]) torch.Size([16]) torch.Size([16, 384]) torch.Size([16]) torch.Size([16])
torch.Size([16, 384]) torch.Size([16, 384]) torch.Size([16, 384]) torch.Size([16]) torch.Size([16, 384]) torch.Size([16]) torch.Size([16])
torch.Size([16, 384]) torch

### Train

In [11]:
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased').cuda()
model.train

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

RuntimeError: CUDA error: all CUDA-capable devices are busy or unavailable

In [8]:
# Optimizer와 Loss 함수는 가장 일반적인 것으로 정의했다.
# 이 노트북 파일의 목적은 BERT를 이용해서 높은 성능의 모델을 간편하게 만들 수 있다는 것을 보여주기 위함이다.
# Optimizer와 Loss를 최적화할 경우 좋은 성능이 나온 이유를 잘 설명할 수 없다.
optimizer = AdamW(model.parameters(), lr = 2e-5, eps = 1e-8)
loss = nn.CrossEntropyLoss()

In [9]:
n_epoch = 10

In [10]:
def train(model, dataloader, optimizer):
    tbar = tqdm(dataloader, desc='Training', leave=True)
    
    total_loss = 0.0
    for i, batch in enumerate(tbar):
        optimizer.zero_grad()
        
        # cls_index와 p_mask는 XLNet 모델에 사용되므로 BERT에서는 사용하지 않는다.
        input_ids, input_mask, segment_ids, cls_index, p_mask, start_positions, end_positions = batch
        
        # to cuda
        input_ids = input_ids.cuda()
        input_mask = input_mask.cuda()
        segment_ids = segment_ids.cuda()
        start_positions = start_positions.cuda()
        end_positions = end_positions.cuda()
        
        # train model
        #out = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        inputs = {
            'input_ids': input_ids,
            'token_type_ids': segment_ids,
            'attention_mask': input_mask,
        }
        out = model(**inputs, start_positions=start_positions, end_positions=end_positions)
        loss = out.loss

        #print('before backward: {}'.format(loss))
        loss.backward()
        #torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        #print('after backward: {}'.format(loss))
        
        optimizer.step()
        
        total_loss += loss.data.item()
        tbar.set_description("Average Loss = {:.4f})".format(total_loss/(i+1)))
        

In [11]:
for i in range(n_epoch):
    train(model, train_dataloader, optimizer)

Average Loss = 0.0497): 100%|██████████| 8247/8247 [1:24:43<00:00,  1.62it/s]


In [96]:
torch.save(model.state_dict(), 'squad_model.bin')

In [16]:
train(model, train_dataloader, optimizer)

Average Loss = 0.0495): 100%|██████████| 8247/8247 [1:24:22<00:00,  1.63it/s]


In [15]:
! ls -alh *.bin

-rw-r--r-- 1 jkfirst deep-learners 416M  6월 23 23:28 squad_model.bin


### Inference

In [ ]:
앞에서는 BERT를 이용해서 분류 모델을 만들었다. 이번 절에서는 BERT를 이용해서 질의응답 모델을 만들어보자. 앞 절에서와 마찬가지로 질의응답 모델을 만들기 위한 데이터셋과 데이터로더에 대한 설명은 생략하고, 학습에 사용할 데이터셋에 대한 설명과 질의응답 모델의 구조에 대해서 이야기해볼 예정이다. 

학습에 사용할 데이터셋은 SQuAD 2.0 데이터셋 이다. SQuAD는 Stanford Question Answering Dataset의 약자로 스텐포드 대학교에서 만든 질의응답 데이터셋이다. SQuAD 2.0 데이터셋은 하나의 문단과 그 문단에 대한 여러 개의 <질문, 답> 쌍으로 이루어져 있다. 데이터셋의 예시는 아래와 같다.
<블록 시작>
{
    "qas": [
		{"question": "When did Beyonce start becoming popular?", "id": "56be85543aeaaa14008c9063", "answers": [{"text": "in the late 1990s", "answer_start": 269}], "is_impossible": false},
		{"question": "What areas did Beyonce compete in when she was growing up?", "id": "56be85543aeaaa14008c9065", "answers": [{"text": "singing and dancing", "answer_start": 207}], "is_impossible": false},
		{"question": "When did Beyonce leave Destiny's Child and become a solo singer?", "id": "56be85543aeaaa14008c9066", "answers": [{"text": "2003", "answer_start": 526}], "is_impossible": false},
		{"question": "In what city and state did Beyonce  grow up? ", "id": "56bf6b0f3aeaaa14008c9601", "answers": [{"text": "Houston, Texas", "answer_start": 166}], "is_impossible": false},
		{"question": "In which decade did Beyonce become famous?", "id": "56bf6b0f3aeaaa14008c9602", "answers": [{"text": "late 1990s", "answer_start": 276}], "is_impossible": false},
		...
		{"question": "What was the name of Beyonc\u00e9's first solo album?", "id": "56d43ce42ccc5a1400d830b5", "answers": [{"text": "Dangerously in Love", "answer_start": 505}], "is_impossible": false}],
    "context": "Beyonc\u00e9 Giselle Knowles-Carter (/bi\u02d0\u02c8j\u0252nse\u026a/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyonc\u00e9's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles \"Crazy in Love\" and \"Baby Boy\"."
}
<블록 끝>

SQuAD 데이터셋은 json 파일 포멧으로 이루어져 있다. "qas"는 질의 응답 쌍 여러 개를 리스트 형태로 가지고 있고 그 질의 응답 쌍은 "context"로부터 추출한 것이다. "qas"는 "question", "id", "answers", "is_impossible" 키를 가지고 있다. "question"은 질문, "id"는 식별을 위한 ID값, "answers"은 질문에 대한 답이고, "is_impossible"은 "context"로부터 알아낼 수 있는 답인지에 대한 참/거짓 값이다. 가령 "context"로부터 답을 할 수 없는 질문에 경우 is_impossible은 true 값을 가진다. "answers"은 다시 "text"와 "answer_start"를 키로 갖는다. "text"는 질문에 대한 답을 "context"에서 찾아서 텍스트로 표현한 것이고, "answer_start"는 "context"에서 "answers"가 위치하는 인덱스의 시작값이다. 예를 들어서 위의 예시에서 {"text": "in the late 1990s", "answer_start": 269} 부분을 보면 "in the late 1990s"의 첫번째 문자인 "i"가 "context"의 269번째 문자에서 시작한다는 것이다.

SQuAD